## Visualize KUKA Arm

In [1]:
from pydrake.geometry import (
    ClippingRange,
    ColorRenderCamera,
    DepthRange,
    DepthRenderCamera,
    MakeRenderEngineVtk,
    RenderCameraCore,
    RenderEngineVtkParams,
    RenderLabel,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import BodyIndex
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.controllers import InverseDynamicsController
from pydrake.systems.primitives import ConstantVectorSource

import copy
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from pydrake.visualization import AddDefaultVisualization

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

iiwa_url = (
   "package://drake/manipulation/models/iiwa_description/sdf/"
   "iiwa14_no_collision.sdf"
)
(iiwa_instance,) = Parser(plant).AddModels(url=iiwa_url)
plant.WeldFrames(
    frame_on_parent_F=plant.world_frame(),
    frame_on_child_M=plant.GetFrameByName("iiwa_link_0", iiwa_instance),
)
plant.Finalize()

for i in range(8):
    link_str = f"iiwa_link_{i}"
    AddMultibodyTriad(plant.GetFrameByName(link_str), scene_graph)

kp, ki, kd = np.zeros(7), np.zeros(7), np.zeros(7)
kp[:] = 0.3
controller = builder.AddSystem(InverseDynamicsController(plant, kp, ki, kd, False))
    
desired_state_input = np.array([[-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1]]).T
builder.AddSystem(ConstantVectorSource(desired_state_input))

desired_state_input = np.array([[-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1]]).T
desired_state_source = builder.AddSystem(ConstantVectorSource(desired_state_input))
desired_state_source.set_name("desired_state_constant")

builder.Connect(plant.get_state_output_port(), controller.get_input_port_estimated_state())
builder.Connect(controller.get_output_port_control(), plant.get_actuation_input_port(iiwa_instance))
builder.Connect(desired_state_source.get_output_port(), controller.get_input_port_desired_state())

renderer_name = "renderer"
scene_graph.AddRenderer(
    renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

In [4]:
AddDefaultVisualization(builder=builder, meshcat=meshcat)
diagram = builder.Build()
simulation = Simulator(diagram)
simulation.Initialize()

In [7]:
simulation.AdvanceTo(300)